In [1]:
import warnings
import xarray
import intake
import cmocean as cm
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from xmovie import Movie
import matplotlib.path as mpath
import xarray as xr
import cartopy.feature as cft
import numpy as np
import cosima_cookbook as cc
import pandas as pd

pd.set_option('display.max_columns', None)

warnings.filterwarnings("ignore") # Suppress warnings for these docs

In [2]:
from dask.distributed import Client

client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 7,Total memory: 256.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39715,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 7
Started: Just now,Total memory: 256.00 GiB
Comm: tcp://127.0.0.1:35385,Total threads: 1
Dashboard: /proxy/36347/status,Memory: 36.57 GiB
Nanny: tcp://127.0.0.1:38575,


In [3]:
import cosima_cookbook as cc
db = 'local_cc_scratch_Ryan_data.db'
session = cc.database.create_session(db)

In [4]:
### Load in wind stress data

# tau_y = cc.querying.getvar(expt='025deg_era5_iaf_1958cycle1', variable='tau_y', 
#                           session=session, frequency='1 monthly',
#                           attrs={'cell_methods': 'time: mean'},
#                           chunks={'time':-1,'xu_ocean':'300MB','yu_ocean':'300MB'})
# tau_y

tau_y = cc.querying.getvar(expt='025deg_jra55_iaf_cycle6_online_mlt', variable='tau_y', 
                           session=session, frequency='1 monthly',
                           attrs={'cell_methods': 'time: mean'},
                           chunks={'time':-1,'xu_ocean':'300MB','yu_ocean':'300MB'}).\
                            sel(time = slice('1982','2023'))

tau_y

<xarray.DataArray 'tau_y' (time: 504, yu_ocean: 1080, xu_ocean: 1440)> Size: 3GB
dask.array<getitem, shape=(504, 1080, 1440), dtype=float32, chunksize=(12, 1080, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 4kB 1982-01-14T12:00:00 ... 2023-12-14T12...
  * xu_ocean  (xu_ocean) float64 12kB -279.8 -279.5 -279.2 ... 79.5 79.75 80.0
  * yu_ocean  (yu_ocean) float64 9kB -81.02 -80.92 -80.81 ... 89.79 89.89 90.0
Attributes:
    long_name:      j-directed wind stress forcing v-velocity
    units:          N/m^2
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  surface_downward_y_stress
    ncfiles:        ['/g/data/e14/rmh561/access-om2/archive/025deg_jra55_iaf_...
    contact:        Ryan Holmes
    email:          ryan.holmes@unsw.edu.au
    created:        2020-11-03
    description:    0.25 degree ACCESS-OM2 global model configuration under i...

In [5]:
import xarray as xr
tau_y_subset = tau_y.sel(time=slice("1982-01-01", "2011-12-31"))

clim_y = tau_y_subset.groupby("time.month").mean(dim="time", keep_attrs=True)

In [6]:
# tau_x = cc.querying.getvar(expt='025deg_era5_iaf_1958cycle1', variable='tau_x', 
#                           session=session, frequency='1 monthly',
#                           attrs={'cell_methods': 'time: mean'},
#                           chunks={'time':-1,'xu_ocean':'300MB','yu_ocean':'300MB'})
# tau_x

tau_x = cc.querying.getvar(
    expt='025deg_jra55_iaf_cycle6_online_mlt',
    variable='tau_x',
    session=session,
    frequency='1 monthly',
    attrs={'cell_methods': 'time: mean'},
    chunks={'time': -1, 'xu_ocean': '300MB', 'yu_ocean': '300MB'}
).sel(time = slice('1982','2023'))

tau_x

<xarray.DataArray 'tau_x' (time: 504, yu_ocean: 1080, xu_ocean: 1440)> Size: 3GB
dask.array<getitem, shape=(504, 1080, 1440), dtype=float32, chunksize=(12, 1080, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 4kB 1982-01-14T12:00:00 ... 2023-12-14T12...
  * xu_ocean  (xu_ocean) float64 12kB -279.8 -279.5 -279.2 ... 79.5 79.75 80.0
  * yu_ocean  (yu_ocean) float64 9kB -81.02 -80.92 -80.81 ... 89.79 89.89 90.0
Attributes:
    long_name:      i-directed wind stress forcing u-velocity
    units:          N/m^2
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  surface_downward_x_stress
    ncfiles:        ['/g/data/e14/rmh561/access-om2/archive/025deg_jra55_iaf_...
    contact:        Ryan Holmes
    email:          ryan.holmes@unsw.edu.au
    created:        2020-11-03
    description:    0.25 degree ACCESS-OM2 global model configuration under i...

In [7]:
import xarray as xr
tau_x_subset = tau_x.sel(time=slice("1982-01-01", "2011-12-31"))

clim_x = tau_x_subset.groupby("time.month").mean(dim="time", keep_attrs=True)

In [8]:
mag_clim = np.sqrt(clim_x**2 + clim_y**2)
mag_clim

<xarray.DataArray (month: 12, yu_ocean: 1080, xu_ocean: 1440)> Size: 75MB
dask.array<sqrt, shape=(12, 1080, 1440), dtype=float32, chunksize=(12, 1080, 1440), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 12kB -279.8 -279.5 -279.2 ... 79.5 79.75 80.0
  * yu_ocean  (yu_ocean) float64 9kB -81.02 -80.92 -80.81 ... 89.79 89.89 90.0
  * month     (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12

# Create Figures from Supplementary for Ross and Weddell Wind Stress

## Fig A4 – Weddell Sea

In [10]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cft
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import string
# import matplotlib.colors as mcolors # No longer needed for quiver

# Set global font
plt.rcParams['font.family'] = 'DejaVu Sans'

# --- SETUP FOR MULTI-PANEL PLOT ---
# Define the months to plot, same as the original script
months_to_plot = pd.date_range("2016-11-01", "2017-02-01", freq="MS")
n_panels = len(months_to_plot)

# Define the grid layout (2x2 for 4 months)
ncols = 2
nrows = (n_panels + ncols - 1) // ncols

# Create the figure and grid of subplots
fig, axes = plt.subplots(
    nrows=nrows, ncols=ncols,
    figsize=(14, 10),
    subplot_kw={'projection': ccrs.Mercator(central_longitude=-20)}
)
axes = axes.flatten()

# Shared plotting parameters
levels = np.linspace(-0.15, 0.15, 61)
s = 18  # Subsampling rate

for i, t in enumerate(months_to_plot):
    ax = axes[i]

    tau_x_month = tau_x.sel(time=t, method="nearest")
    tau_y_month = tau_y.sel(time=t, method="nearest")
    mag_year = np.sqrt(tau_x_month**2 + tau_y_month**2)
    mag_clim_month = mag_clim.sel(month=t.month)
    mag_anom = mag_year - mag_clim_month

    # Weddell Sea
    ax.set_extent([-70, 30, -72, -55], crs=ccrs.PlateCarree())
    ax.add_feature(cft.LAND, facecolor='lightgray', zorder=2)
    ax.coastlines(resolution='50m', zorder=3)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1,
                      color='gray', alpha=0.5, linestyle='--', zorder=1)

    gl.top_labels = False
    gl.right_labels = False
    gl.left_labels = (i % ncols == 0) # Only label the left column
    gl.bottom_labels = (i >= n_panels - ncols) # Only label the bottom row
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 12}
    gl.ylabel_style = {'size': 12}

    # Plot
    mesh_plot = ax.pcolormesh(mag_anom.xu_ocean, mag_anom.yu_ocean, mag_anom,
                              cmap='RdBu_r', transform=ccrs.PlateCarree(),
                              vmin=levels.min(), vmax=levels.max())

    # Subsample vectors
    tau_x_sub = tau_x_month.values[::s, ::s]
    tau_y_sub = tau_y_month.values[::s, ::s]
    
    q = ax.quiver(tau_x_month.xu_ocean[::s], tau_x_month.yu_ocean[::s],
                  tau_x_sub, tau_y_sub, scale=1.6, color='black',
                  transform=ccrs.PlateCarree(),
                  width=0.003, alpha=0.9, zorder=4)

    # Quiver key (reference vector) for each panel
    ax.quiverkey(q, X=0.85, Y=1.05, U=0.2, label='0.2 N/m²', labelpos='E',
                 coordinates='axes', fontproperties={'size': 10})

    panel_label = string.ascii_lowercase[i]
    title = f"{panel_label}. {t.strftime('%B %Y')}"
    ax.set_title(title, loc='left', fontsize=14, fontweight='bold')

cbar_anom_ax = fig.add_axes([0.15, 0.06, 0.7, 0.025])
cbar_anom = fig.colorbar(mesh_plot, cax=cbar_anom_ax, orientation='horizontal', extend='both')
cbar_anom.set_label("Wind Stress Anomaly (N/m²)", fontsize=12)
cbar_anom.ax.tick_params(labelsize=12)

plt.suptitle("Weddell Sea Wind Stress Anomaly & Vectors", fontsize=18, fontweight='bold', y=0.98)

plt.subplots_adjust(
    top=0.92,
    bottom=0.12, 
    left=0.08,
    right=0.95,
    hspace=0.15,
    wspace=0.05
)

output_path = "/g/data/e14/ts7341/wind/ws/weddell_sea_wind_stress_multi_panel_black_vectors.png"
plt.savefig(output_path, dpi=600, bbox_inches='tight')

plt.show()

Saved figure: /g/data/e14/ts7341/wind/ws/weddell_sea_wind_stress_multi_panel_black_vectors.png


## Fig A6 – Ross Sea

In [9]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cft
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import string

# Set global font
plt.rcParams['font.family'] = 'DejaVu Sans'

months_to_plot = pd.date_range("2021-11-01", "2022-02-01", freq="MS")
n_panels = len(months_to_plot)

# Define the grid layout (2x2 for 4 months)
ncols = 2
nrows = (n_panels + ncols - 1) // ncols

fig, axes = plt.subplots(
    nrows=nrows, ncols=ncols,
    figsize=(14, 10),
    subplot_kw={'projection': ccrs.Mercator(central_longitude=180)}
)
axes = axes.flatten()

# Shared plotting parameters
levels = np.linspace(-0.15, 0.15, 61)
s = 18  # Subsampling rate

# Loop each month
for i, t in enumerate(months_to_plot):
    ax = axes[i]

    # Select correct month
    tau_x_month = tau_x.sel(time=t, method="nearest")
    tau_y_month = tau_y.sel(time=t, method="nearest")
    mag_year = np.sqrt(tau_x_month**2 + tau_y_month**2)
    mag_clim_month = mag_clim.sel(month=t.month)
    mag_anom = mag_year - mag_clim_month

    # Map
    ax.set_extent([-200, -130, -78, -65], crs=ccrs.PlateCarree())
    ax.add_feature(cft.LAND, facecolor='lightgray', zorder=2)
    ax.coastlines(resolution='50m', zorder=3)

    # Gridlines that show what I want
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1,
                      color='gray', alpha=0.5, linestyle='--', zorder=1)
    gl.xlocator = mticker.FixedLocator([160, 180, -160, -140])
    gl.ylocator = mticker.FixedLocator([-75, -70, -65])
    gl.top_labels = False
    gl.right_labels = False
    gl.left_labels = (i % ncols == 0)
    gl.bottom_labels = (i >= n_panels - ncols)
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 12}
    gl.ylabel_style = {'size': 12}

    # Plot data
    contour = ax.contourf(mag_anom.xu_ocean, mag_anom.yu_ocean, mag_anom,
                          levels=levels, cmap='RdBu_r', transform=ccrs.PlateCarree(),
                          extend='both')

    tau_x_sub = tau_x_month.values[::s, ::s]
    tau_y_sub = tau_y_month.values[::s, ::s]
    # mag_sub = np.sqrt(tau_x_sub**2 + tau_y_sub**2) 

    # Make sure you have quivers
    q = ax.quiver(tau_x_month.xu_ocean[::s], tau_x_month.yu_ocean[::s],
                  tau_x_sub, tau_y_sub, scale=1.6, color='black',
                  transform=ccrs.PlateCarree(),
                  width=0.003, alpha=0.9, zorder=4)

    # Quiver key (reference vector) for each panel
    ax.quiverkey(q, X=0.85, Y=1.05, U=0.2, label='0.2 N/m²', labelpos='E',
                 coordinates='axes', fontproperties={'size': 10})

    panel_label = string.ascii_lowercase[i]
    title = f"{panel_label}. {t.strftime('%B %Y')}"
    ax.set_title(title, loc='left', fontsize=14, fontweight='bold')

# Colourbars
cbar_anom_ax = fig.add_axes([0.15, 0.06, 0.7, 0.025])
cbar_anom = fig.colorbar(contour, cax=cbar_anom_ax, orientation='horizontal', extend='both')
cbar_anom.set_label("Wind Stress Anomaly (N/m²)", fontsize=12)
cbar_anom.ax.tick_params(labelsize=12)

plt.suptitle("Ross Sea Wind Stress Anomaly & Vectors", fontsize=18, fontweight='bold', y=0.98)


plt.subplots_adjust(
    top=0.92,
    bottom=0.12,  # REDUCED: Closes gap from removed colorbar.
    left=0.08,
    right=0.95,
    hspace=0.15,
    wspace=0.05
)

output_path = "/g/data/e14/ts7341/wind/ross_sea_wind_stress_multi_panel_fixed_black_vectors.png"
plt.savefig(output_path, dpi=600, bbox_inches='tight')

plt.show()

Saved figure: /g/data/e14/ts7341/wind/ross_sea_wind_stress_multi_panel_fixed_black_vectors.png
